# Multi-factor selection with IC method

## 1. Steps of multi-factor selection

1. 因子预处理：将候选因子进行预处理，去极值、标准化，以确保因子在同一数量级下可比较。
2. 因子打分：使用每个因子对股票进行打分，得到每个股票在每个因子上的得分。
3. 因子合成：将每个因子的得分加权合成为一个综合得分,使用等权重方法。
4. 分组排序：将股票按照综合得分从高到低排序，并将它们分为5个等权重的组。
5. IC计算：计算每个组之问的平均绩效差异，即每个组的平均绩效减去前一个组的平均绩效，得到每个组之问的IC值。
6. IC校验：对IC值进行显著性检验，以确定因子是否具有预测股票绩效的能力。
7. 因子选择：根据IC值和显著性检验结果，选择最优的因子集合，并使用这些因子进行多因子选股。你能帮我用python实现一下吗？

## 2. 具体实验操作
### 2.1 导入需要的包

In [ ]:
import rqalpha as rq
import pandas as pd
import numpy as np

### 2.2 数据获取和预处理

使用米筐的API获取沪深300指数成分股的历史价格和所有因子数据，并将其合并到一个数据框中进行进一步处理。

In [ ]:
# 开始编写你的主要的算法逻辑

start_date = '20221201'
end_date = '20221231'
universe = '000001.XSHE'

# 获取股票历史价格数据
price_arr = history_bars(universe, 22, '1d', ['close'])
price_arr= price_arr['close']
price_df = pd.DataFrame(price_arr, columns = ['close'])

factor_list = rqc.get_all_factor_names()
factor_dict = rqc.get_factor(universe,factor=factor_list[:100],start_date=start_date,end_date=end_date)

# 将所有因子数据合并到一个数据框中
factor_df = pd.DataFrame(index=price_df.index)

for factor_name in factor_dict.keys():
    # factor_data is series
    factor_data = factor_dict[factor_name]
    factor_data = factor_data.reset_index().drop_duplicates(['order_book_id','date'], keep='last')
    factor_data = factor_data.pivot(index='date', columns='order_book_id')
    factor_data.columns = [factor_name]
    factor_data.index = price_df.index
    factor_df = pd.concat([factor_df, factor_data], axis=1)

# 去极值和标准化处理
factor_df = factor_df.clip(lower=factor_df.quantile(0.01), upper=factor_df.quantile(0.99), axis=1)
factor_df = (factor_df - factor_df.mean()) / factor_df.std()


### 2.3 因子打分和因子合成

使用每个因子对股票进行打分，并将它们合成为一个综合得分。

In [ ]:
# 使用每个因子对股票进行打分，并将得分归一化到 0-1 范围
factor_score_df = (factor_df.rank(pct=True) * 100).apply(np.floor) / 100

# 计算每个股票的综合得分，并使用等权重方法合成
score_df = factor_score_df.mean(axis=1)




### 2.4 分组排序

将股票按照综合得分从高到低排序，并将它们分为5个等权重的组。

In [ ]:
# 将股票按综合得分从高到低排序
score_df = score_df.sort_values(ascending=False)
# 分为5个等权重的组
group_num = 5
group_size = len(score_df) // group_num
group_list = []
for i in range(group_num - 1):
    group_list.append(score_df.iloc[i * group_size: (i + 1) * group_size])
group_list.append(score_df.iloc[(group_num - 1) * group_size:])



### 2.5 IC计算

计算每个组之间的平均绩效差异，即每个组的平均绩效减去前一个组的平均绩效，并得到每个组之间的IC值。

In [ ]:
# 计算每个组的平均绩效
perf_df = price_df.pct_change().shift(-1)
perf_mean_list = []
# print(perf_df)
for group in group_list:
    group_perf = perf_df.loc[group.index].mean(axis=1)
    perf_mean_list.append(group_perf.mean())

# 计算每个组之间的IC值
ic_list = []
for i in range(group_num - 1):
    ic = perf_mean_list[i + 1] - perf_mean_list[i]
    ic_list.append(ic)


### 2.6 IC校验

对IC值进行显著性检验，以确定因子是否具有预测股票绩效的能力。这里我们采用t检验进行检验，显著性水平为0.05。

In [ ]:
from scipy.stats import ttest_ind

# 进行t检验
t, p = ttest_ind(ic_list, np.zeros(len(ic_list)), equal_var=False)

# 显示检验结果
if p < 0.05:
    print("IC值显著不为0，因子具有预测股票绩效的能力。")
else:
    print("IC值不显著为0，因子不具有预测股票绩效的能力。")


### 2.7 因子选择

根据IC值和显著性检验结果，选择最优的因子集合，并使用这些因子进行多因子选股。这里我们选择IC值最大的前5个因子进行选股。

In [ ]:
# 选择IC值最大的前5个因子
selected_factors = factor_df.corrwith(score_df).abs().nlargest(5).index.tolist()

print("选定的因子为：", selected_factors)

# 使用选定的因子进行多因子选股
selected_stock = factor_df[selected_factors].rank(pct=True).mean(axis=1).nlargest(10).index.tolist()
print("选定的股票为：", selected_stock)